# Digit Recognizer
## I will be using Tensorflow to create a neural network that learns to understand numbers

In [ ]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import theano
import theano.tensor as T

# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline

In [ ]:
# Read from csv
data = []
with open("train.csv", "rb") as csvfile:
    trainreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(trainreader)
    for row in trainreader:
        data.append(row)
data = np.array(data, dtype=float)

In [ ]:
# Normalize data
X = data[:, 1:]
mu = np.mean(X, axis=0)
sigma = np.std(X, axis=0)
data[:, 1:] = np.nan_to_num((X-mu)/sigma)
np.random.shuffle(data)

In [ ]:
train = data[:33600]
cv = data[33600:]

In [ ]:
nn_input_dim = train[:, 1:].shape[1]
nn_output_dim = 10
num_examples = train.shape[0]

In [ ]:
Xvec = T.matrix('X')
yvec = T.lvector('y')

In [ ]:
nn_hdim = 3
lmbda = 0.5
epsilon = 0.5

In [ ]:
W1 = theano.shared(np.random.randn(nn_input_dim, nn_hdim), name='W1')
b1 = theano.shared(np.zeros(nn_hdim), name='b1')
W2 = theano.shared(np.random.randn(nn_hdim, nn_output_dim), name='W2')
b2 = theano.shared(np.zeros(nn_output_dim), name='b2')

### Feed-forward Propogation

In [ ]:
z1 = Xvec.dot(W1) + b1
a1 = T.tanh(z1)
z2 = a1.dot(W2) + b2
prob = T.nnet.softmax(z2)

prediction = T.argmax(prob, axis=1)

### Cost Function

In [ ]:
reg_loss = 1./num_examples * lmbda/2 * (T.sum(T.sqr(W1)) + T.sum(T.sqr(W2)))
loss = T.nnet.categorical_crossentropy(prob, yvec).mean() + reg_loss

### Back Propogation

In [ ]:
dW1 = T.grad(loss, W1)
db1 = T.grad(loss, b1)
dW2 = T.grad(loss, W2)
db2 = T.grad(loss, b2)

### Functions

In [ ]:
forward_prop = theano.function([Xvec], prob)
calculate_loss = theano.function([Xvec, yvec], loss)
predict = theano.function([Xvec], prediction)

gradient_step = theano.function(
    [Xvec, yvec], loss,
    updates=((W2, W2 - epsilon * dW2),
             (W1, W1 - epsilon * dW1),
             (b2, b2 - epsilon * db2),
             (b1, b1 - epsilon * db1)))

## Train function

In [ ]:
def train_func(it=5000, printloss=False):
    W1 = theano.shared(np.random.randn(nn_input_dim, nn_hdim), name='W1')
    b1 = theano.shared(np.zeros(nn_hdim), name='b1')
    W2 = theano.shared(np.random.randn(nn_hdim, nn_output_dim), name='W2')
    b2 = theano.shared(np.zeros(nn_output_dim), name='b2')

    for i in range(it):
        Xtrain, ytrain = train[:num_examples, 1:], train[:num_examples, 0]
        Xcv, ycv = cv[:, 1:], cv[:, 0]

        train_cost = gradient_step(Xtrain, np.array(ytrain, dtype=int))
        
        if printloss and i%1000 == 0:
            print("The loss at %d with %d example(s) is %f" % (i, num_examples, train_cost))

        cv_cost = calculate_loss(Xcv, np.array(ycv, dtype=int))
    return train_cost, cv_cost

### Run #1 with the following parameters:

nn_hdim = 3; lmbda = 0.5; epsilon = 0.5

In [ ]:
train_cost_list = []
cv_cost_list = []

for i in range(1, train.shape[0], 1000):
    num_examples = i
    train_cost, cv_cost = train_func(2001)
    train_cost_list.append(train_cost)
    cv_cost_list.append(cv_cost)

In [ ]:
plt.xlabel("m")
plt.ylabel("Cost")
plt.plot(train_cost_list, c="r", label="Training Cost")
plt.plot(cv_cost_list, c="g", label="Cross Validation Cost")
plt.show()